<a href="https://colab.research.google.com/github/Ahnkyuwon504/AI-modeling/blob/main/eval-rag-and-instruct/gemma_ko_2b_rag_pymongo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 라이브러리 설치

In [1]:
!pip -q install datasets pandas pymongo sentence_transformers
!pip -q install -U transformers
!pip -q install accelerate
!pip -q "pymongo[srv]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 383.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 62.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu

# 2. Dataset

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [29]:
from datasets import load_dataset
import pandas as pd

# https://huggingface.co/datasets/AIatMongoDB/embedded_movies
dataset = load_dataset("AIatMongoDB/embedded_movies")

# dataframe으로 변환
dataset_df = pd.DataFrame(dataset["train"])

dataset_df.head(5)

,plot,genres,runtime,cast,num_mflix_comments,poster,title,fullplot,languages,directors,writers,awards,imdb,countries,type,plot_embedding,rated,metacritic
0,Young Pauline is left a lot of money when her ...,[Action],199.0,"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",0,https://m.media-amazon.com/images/M/MV5BMzgxOD...,The Perils of Pauline,Young Pauline is left a lot of money when her ...,[English],"[Louis J. Gasnier, Donald MacKenzie]","[Charles W. Goddard (screenplay), Basil Dickey...","{'nominations': 0, 'text': '1 win.', 'wins': 1}","{'id': 4465, 'rating': 7.6, 'votes': 744}",[USA],movie,"[0.0007293965299999999, -0.026834568000000003,...",None,NaN
1,A penniless young man tries to save an heiress...,"[Comedy, Short, Action]",22.0,"[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",0,https://m.media-amazon.com/images/M/MV5BNzE1OW...,From Hand to Mouth,As a penniless man worries about how he will m...,[English],"[Alfred J. Goulding, Hal Roach]",[H.M. Walker (titles)],"{'nominations': 1, 'text': '1 nomination.', 'w...","{'id': 10146, 'rating': 7.0, 'votes': 639}",[USA],movie,"[-0.022837115, -0.022941574000000003, 0.014937...",TV-G,NaN
2,"Michael ""Beau"" Geste leaves England in disgrac...","[Action, Adventure, Drama]",101.0,"[Ronald Colman, Neil Hamilton, Ralph Forbes, A...",0,None,Beau Geste,"Michael ""Beau"" Geste leaves England in disgrac...",[English],[Herbert Brenon],"[Herbert Brenon (adaptation), John Russell (ad...","{'nominations': 0, 'text': '1 win.', 'wins': 1}","{'id': 16634, 'rating': 6.9, 'votes': 222}",[USA],movie,"[0.00023330492999999998, -0.028511643000000003...",None,NaN
3,"Seeking revenge, an athletic young man joins t...","[Adventure, Action]",88.0,"[Billie Dove, Tempe Pigott, Donald Crisp, Sam ...",1,https://m.media-amazon.com/images/M/MV5BMzU0ND...,The Black Pirate,A nobleman vows to avenge the death of his fat...,None,[Albert Parker],"[Douglas Fairbanks (story), Jack Cunningham (a...","{'nominations': 0, 'text': '1 win.', 'wins': 1}","{'id': 16654, 'rating': 7.2, 'votes': 1146}",[USA],movie,"[-0.005927917, -0.033394486, 0.0015323418, -0....",None,NaN
4,An irresponsible young millionaire changes his...,"[Action, Comedy, Romance]",58.0,"[Harold Lloyd, Jobyna Ralston, Noah Young, Jim...",0,https://m.media-amazon.com/images/M/MV5BMTcxMT...,For Heaven's Sake,"The Uptown Boy, J. Harold Manners (Lloyd) is a...",[English],[Sam Taylor],"[Ted Wilde (story), John Grey (story), Clyde B...","{'nominations': 1, 'text': '1 nomination.', 'w...","{'id': 16895, 'rating': 7.6, 'votes': 918}",[USA],movie,"[-0.0059373598, -0.026604708, -0.0070914757000...",PASSED,NaN


In [30]:
print(dataset_df.shape)

dataset_df = dataset_df.dropna(subset=["fullplot"])
print("\n결측치 제거 후:")
print(dataset_df.isnull().sum())

# 기존 임베딩 제거
dataset_df = dataset_df.drop(columns=["plot_embedding"])

(1500, 18)

결측치 제거 후:
plot                    0
genres                  0
runtime                14
cast                    1
num_mflix_comments      0
poster                 78
title                   0
fullplot                0
languages               1
directors              12
writers                13
awards                  0
imdb                    0
countries               0
type                    0
plot_embedding          1
rated                 279
metacritic            893
dtype: int64


In [10]:
# 테스트를 위해 데이터 볼륨 축소
# dataset_df = dataset_df[:100]

In [31]:
from sentence_transformers import SentenceTransformer

# https://huggingface.co/thenlper/gte-large
embedding_model = SentenceTransformer("thenlper/gte-large")

def get_embedding(text: str) -> list[float]:
    if not text.strip():
        print("Attempted to get embedding for empty text.")
        return []

    embedding = embedding_model.encode(text)
    return embedding.tolist()


dataset_df["embedding"] = dataset_df["fullplot"].apply(get_embedding)

dataset_df.head()

,plot,genres,runtime,cast,num_mflix_comments,poster,title,fullplot,languages,directors,writers,awards,imdb,countries,type,rated,metacritic,embedding
0,Young Pauline is left a lot of money when her ...,[Action],199.0,"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",0,https://m.media-amazon.com/images/M/MV5BMzgxOD...,The Perils of Pauline,Young Pauline is left a lot of money when her ...,[English],"[Louis J. Gasnier, Donald MacKenzie]","[Charles W. Goddard (screenplay), Basil Dickey...","{'nominations': 0, 'text': '1 win.', 'wins': 1}","{'id': 4465, 'rating': 7.6, 'votes': 744}",[USA],movie,None,NaN,"[-0.009285837411880493, -0.005062109790742397,..."
1,A penniless young man tries to save an heiress...,"[Comedy, Short, Action]",22.0,"[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",0,https://m.media-amazon.com/images/M/MV5BNzE1OW...,From Hand to Mouth,As a penniless man worries about how he will m...,[English],"[Alfred J. Goulding, Hal Roach]",[H.M. Walker (titles)],"{'nominations': 1, 'text': '1 nomination.', 'w...","{'id': 10146, 'rating': 7.0, 'votes': 639}",[USA],movie,TV-G,NaN,"[-0.0024393806234002113, 0.02309592440724373, ..."
2,"Michael ""Beau"" Geste leaves England in disgrac...","[Action, Adventure, Drama]",101.0,"[Ronald Colman, Neil Hamilton, Ralph Forbes, A...",0,None,Beau Geste,"Michael ""Beau"" Geste leaves England in disgrac...",[English],[Herbert Brenon],"[Herbert Brenon (adaptation), John Russell (ad...","{'nominations': 0, 'text': '1 win.', 'wins': 1}","{'id': 16634, 'rating': 6.9, 'votes': 222}",[USA],movie,None,NaN,"[0.012204288505017757, -0.011455747298896313, ..."
3,"Seeking revenge, an athletic young man joins t...","[Adventure, Action]",88.0,"[Billie Dove, Tempe Pigott, Donald Crisp, Sam ...",1,https://m.media-amazon.com/images/M/MV5BMzU0ND...,The Black Pirate,A nobleman vows to avenge the death of his fat...,None,[Albert Parker],"[Douglas Fairbanks (story), Jack Cunningham (a...","{'nominations': 0, 'text': '1 win.', 'wins': 1}","{'id': 16654, 'rating': 7.2, 'votes': 1146}",[USA],movie,None,NaN,"[0.00454134214669466, -0.0006100605824030936, ..."
4,An irresponsible young millionaire changes his...,"[Action, Comedy, Romance]",58.0,"[Harold Lloyd, Jobyna Ralston, Noah Young, Jim...",0,https://m.media-amazon.com/images/M/MV5BMTcxMT...,For Heaven's Sake,"The Uptown Boy, J. Harold Manners (Lloyd) is a...",[English],[Sam Taylor],"[Ted Wilde (story), John Grey (story), Clyde B...","{'nominations': 1, 'text': '1 nomination.', 'w...","{'id': 16895, 'rating': 7.6, 'votes': 918}",[USA],movie,PASSED,NaN,"[-0.00222560903057456, 0.011567801237106323, -..."


## 2.1. connect mongodb

### 뻘짓1. 매 런타임마다 colab의 호스트를 mongodb의 network access에 넣어줘야한다 ;;

In [18]:
!curl ipecho.net/plain

35.185.30.159

In [19]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

uri = "mongodb+srv://spider504:<password>@kyuwon.rpoeioy.mongodb.net/?retryWrites=true&w=majority&appName=kyuwon"

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


### 뻘짓2. 이건 mongodb에 vector index를 생성해 줘야 하는 거다 ..

In [20]:
{
 "fields": [{
     "numDimensions": 1024,
     "path": "embedding",
     "similarity": "cosine",
     "type": "vector"
   }]
}

{'fields': [{'numDimensions': 1024,
   'path': 'embedding',
   'similarity': 'cosine',
   'type': 'vector'}]}

In [21]:
import pymongo
from google.colab import userdata

def get_mongo_client(mongo_uri):
    """Establish connection to the MongoDB."""
    try:
        client = pymongo.MongoClient(mongo_uri)
        print("Connection to MongoDB successful")
        return client
    except pymongo.errors.ConnectionFailure as e:
        print(f"Connection failed: {e}")
        return None

mongo_uri = userdata.get("MONGO_URI")
if not mongo_uri:
    print("MONGO_URI not set in environment variables")

mongo_client = get_mongo_client(mongo_uri)

# Ingest data into MongoDB
db = mongo_client["spider504"]
collection = db["240703_movie_collection"]

Connection to MongoDB successful


각 행을 개별 딕셔너리로 변환 후 적재

In [32]:
collection.delete_many({})

documents = dataset_df.to_dict("records")
collection.insert_many(documents)

print("Data ingestion into MongoDB completed")

Data ingestion into MongoDB completed


# 3. Vector search

In [44]:
# 데이터 확인
print("Inserted Documents:", collection.find_one())

Inserted Documents: {'_id': ObjectId('66855d59fa436c367dbfe27c'), 'plot': "Young Pauline is left a lot of money when her wealthy uncle dies. However, her uncle's secretary has been named as her guardian until she marries, at which time she will officially take ...", 'genres': ['Action'], 'runtime': 199.0, 'cast': ['Pearl White', 'Crane Wilbur', 'Paul Panzer', 'Edward Josè'], 'num_mflix_comments': 0, 'poster': 'https://m.media-amazon.com/images/M/MV5BMzgxODk1Mzk2Ml5BMl5BanBnXkFtZTgwMDg0NzkwMjE@._V1_SY1000_SX677_AL_.jpg', 'title': 'The Perils of Pauline', 'fullplot': 'Young Pauline is left a lot of money when her wealthy uncle dies. However, her uncle\'s secretary has been named as her guardian until she marries, at which time she will officially take possession of her inheritance. Meanwhile, her "guardian" and his confederates constantly come up with schemes to get rid of Pauline so that he can get his hands on the money himself.', 'languages': ['English'], 'directors': ['Louis J. Gasni

In [58]:
def vector_search(user_query, collection):
    """
    Perform a vector search in the MongoDB collection based on the user query.

    Args:
    user_query (str): The user's query string.
    collection (MongoCollection): The MongoDB collection to search.

    Returns:
    list: A list of matching documents.
    """

    # Generate embedding for the user query
    query_embedding = get_embedding(user_query)

    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    # Define the vector search pipeline
    pipeline = [
        {
            "$vectorSearch": {
                "index": "vector_index",
                "queryVector": query_embedding,
                "path": "embedding",
                "numCandidates": 150,  # Number of candidate matches to consider
                "limit": 4,  # Return top 4 matches
            }
        },
        {
            "$project": {
                "_id": 0,  # Exclude the _id field
                "fullplot": 1,  # Include the plot field
                "title": 1,  # Include the title field
                "genres": 1,  # Include the genres field
                "score": {"$meta": "vectorSearchScore"},  # Include the search score
            }
        },
    ]

    results = collection.aggregate(pipeline)

    return list(results)

In [54]:
def get_search_result(query, collection):

    get_knowledge = vector_search(query, collection)

    search_result = ""
    for result in get_knowledge:
        search_result += f"Title: {result.get('title', 'N/A')}, Plot: {result.get('fullplot', 'N/A')}\n"

    return search_result

In [61]:
# Conduct query with retrival of sources
query = "What is the best horror movie to watch and why?"
source_information = get_search_result(query, collection)
combined_information = (
    f"Query: {query}\nContinue to answer the query by using the Search Results:\n{source_information}."
)

print(combined_information)

Query: What is the best horror movie to watch and why?
Continue to answer the query by using the Search Results:
Title: Pet Sematary II, Plot: The "sematary" is up to its old zombie-raising tricks again. This time, the protagonists are Jeff Matthews, whose mother died in a Hollywood stage accident, and Drew Gilbert, a boy coping with an abusive stepfather.
Title: Dawn of the Dead, Plot: Ana, a young beautiful nurse finishes her day-shift at the hospital to return home to her beloved husband, they make love and sleep together. The next day, after her husband is killed by her neighbor next door, he suddenly comes back to life. She discovers the chaos happening in her neighborhood and escapes from her home. Soon after coming to her senses in the woods, she encounters a cop and other survivors, they decide to find safety in a mall. Soon more survivors come, and they learn that if they want to stay alive, they should stick together as the world is overrun by an army of undead. Can they surv

# 4. 추론

In [60]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

<bos>Query: What is the best romantic movie to watch and why?
Continue to answer the query by using the Search Results:
Title: Shut Up and Kiss Me!, Plot: Ryan and Pete are 27-year old best friends in Miami, born on the same day and each searching for the perfect woman. Ryan is a rookie stockbroker living with his psychic Mom. Pete is a slick surfer dude yet to find commitment. Each meets the women of their dreams on the same day. Ryan knocks heads in an elevator with the gorgeous Jessica, passing out before getting her number. Pete falls for the insatiable Tiara, but Tiara's uncle is mob boss Vincent Bublione, charged with her protection. This high-energy romantic comedy asks to what extent will you go for true love?
Title: Pearl Harbor, Plot: Pearl Harbor is a classic tale of romance set during a war that complicates everything. It all starts when childhood friends Rafe and Danny become Army Air Corps pilots and meet Evelyn, a Navy nurse. Rafe falls head over heels and next thing you

In [62]:
input_ids = tokenizer(combined_information, return_tensors="pt").to("cuda")
response = model.generate(**input_ids, max_new_tokens=500)
print(tokenizer.decode(response[0]))

<bos>Query: What is the best horror movie to watch and why?
Continue to answer the query by using the Search Results:
Title: Pet Sematary II, Plot: The "sematary" is up to its old zombie-raising tricks again. This time, the protagonists are Jeff Matthews, whose mother died in a Hollywood stage accident, and Drew Gilbert, a boy coping with an abusive stepfather.
Title: Dawn of the Dead, Plot: Ana, a young beautiful nurse finishes her day-shift at the hospital to return home to her beloved husband, they make love and sleep together. The next day, after her husband is killed by her neighbor next door, he suddenly comes back to life. She discovers the chaos happening in her neighborhood and escapes from her home. Soon after coming to her senses in the woods, she encounters a cop and other survivors, they decide to find safety in a mall. Soon more survivors come, and they learn that if they want to stay alive, they should stick together as the world is overrun by an army of undead. Can they